<a href="https://colab.research.google.com/github/srujanreddyj/recommendation_systems_corise/blob/main/week4_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import dcg_score,ndcg_score
from collections import OrderedDict

# **Welcome to week 4 project!**

Welcome to the last week of the course -- so excited to see that you've made it to the end! 👏 

We’ve already discussed the importance of measuring model performance. As Lord Kelvin said, “To measure is to know – If you cannot measure it, you cannot improve it.” And he was right – metrics are the only way we can actually evaluate our model’s performance!

In this week's project, we will touch upon two key aspects related to evaluation:
1. Behavioral metrics
2. Off-policy evaluation

For behavioral metrics, we will work with Spotify music sessions dataset, and implement a few behavioral metrics and understand their relationships with traditional metrics.

For off-policy evaluation, we will simulate a dataset where we have logged action policies, and see how IPS is implemented.


### Goals for this week's project:

For this week's project assignment, we will complete the following tasks:
1. Implement 3 behavioral metrics and present the correlation plot for them
2. Implement another ranking logic (e.g. sort by track popularity, or sort by danceability and compare all metrics on productional ranking logic and this new ranking logic.
3. Complete the implementation of two off-policy estimators: Capped IPS and Normalized Capped Importance Sampling (NCIS)


# Part A: Behavioral metrics

Behavioral metrics include factors like what items a user interacts with and how, the amount of time they spend on the platform, and how they spend that time.

To define and implement a few behavioral metrics, we will work with the Spotify music sessions dataset.
Download the dataset from GDrive: https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing
(files: log_mini.cvs and tf_mini.csv)

## Spotify music sessions dataset

The public part of the dataset consists of roughly 130 million listening sessions with associated user interactions on the Spotify service. In total, users interacted with almost 4 million tracks during these sessions, and the dataset includes acoustic features and metadata for all of these tracks.

Detailed description of the dataset can be found here:
https://drive.google.com/file/d/1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf/view?usp=sharing



In [2]:
!gdown --folder https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing -O ./

Retrieving folder list
Processing file 1_jFxa2s7ONQ7nrhqFc1lARP6WChs2T4e data1.txt
Processing file 1rEJFi98sLUTYj50Fh8ff33A7NKOoT87c hmdata.zip
Processing file 1D3RyRBKBaw0KbLxpJUQDjVIt5CLpE2n9 log_mini.csv
Processing file 1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf Spotify-dataset-description.pdf
Processing file 1Z8N0Xf9M34PuN4V5G5uAxasvOCm6TVp4 tf_mini.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_jFxa2s7ONQ7nrhqFc1lARP6WChs2T4e
To: /content/data/data1.txt
100% 158M/158M [00:01<00:00, 85.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rEJFi98sLUTYj50Fh8ff33A7NKOoT87c
To: /content/data/hmdata.zip
100% 773M/773M [00:06<00:00, 122MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D3RyRBKBaw0KbLxpJUQDjVIt5CLpE2n9
To: /content/data/log_mini.csv
100% 28.9M/28.9M [00:00<00:00, 77.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BELTuH4nBeyHna5EAGzJv-HWHKrbxPs

In [3]:
log = pd.read_csv("/content/data/log_mini.csv")
tracks = pd.read_csv("/content/data/tf_mini.csv")

In [4]:
log.head(2)

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone


In [5]:
tracks.head(2)

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,...,4,0.935512,-0.033284,-0.411896,-0.02858,0.349438,0.832467,-0.213871,-0.299464,-0.675907
1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,3,0.359675,0.145703,-0.850372,0.12386,0.746904,0.371803,-0.420558,-0.213120,-0.525795


### Traditional metrics: ndcg@k

We will use the log dataframe as the main dataframe for evaluation of metrics. The skip_1 flag can be used as a relevance signal -- if the user found the recommendation relevant, skip_1 = False. With this relevance signal, we can compute simple ndcg metrics -- one for each session and then averaged across all sessions. This will serve as a base metric for comparison.

Note: the ranking logic here is assumed to be the production ranker, i.e. sorting by session_position gives the exact order of tracks the Spotify ranker presented to the user.

Lets compute simple skip rate and ndcg metric for the production ranker:

In [6]:
topk = 10

skip_rate_session = log.groupby('session_id').apply(lambda x: x.nsmallest(topk,['session_position'])).reset_index(drop = True).groupby("session_id").skip_1.mean().mean()
print("average skip rate in the session: ",skip_rate_session)

average skip rate in the session:  0.40894


In [7]:
topk = 10

def get_ndcg(df):
    true_relevance = np.asarray(1-np.asarray(df['skip_1']*1.0))
    ranker_scores = np.asarray(1/np.asarray(df['session_position'])) # approximate the ranker scores using the session position
    return(ndcg_score([true_relevance], [ranker_scores]))


ndcg_metric = log.groupby('session_id').apply(lambda x: x.nsmallest(topk,['session_position'])).reset_index(drop = True).groupby("session_id").apply(get_ndcg).mean()
print("NDCG@k, with k= ",topk," is: ",ndcg_metric)

NDCG@k, with k=  10  is:  0.8330266041453142


In [8]:
def calc_topk_metrics(topk, metric_fn):
    return log.groupby('session_id')\
            .apply(lambda x: x.nsmallest(topk, ['session_position']))\
            .reset_index(drop = True)\
            .groupby("session_id")\
            .apply(metric_fn)\

metrics_ndcg_5 = calc_topk_metrics(topk=5, metric_fn=get_ndcg)
metrics_ndcg_10 = calc_topk_metrics(topk=10, metric_fn=get_ndcg)
print("NDCG@k, with k= 5 is: ", metrics_ndcg_5.mean())
print("NDCG@k, with k= 10 is: ", metrics_ndcg_10.mean())

NDCG@k, with k= 5 is:  0.8235700606177565
NDCG@k, with k= 10 is:  0.8330266041453142


## Goals for this week:

Implement a few behavioral metrics and compare their correlations. We will implement the following three metrics:
1. *Time to first skip:* how long did it take for the user to get the first bad recommendation, i.e. a recommendation they skipped. Since we can't easily calculate time, we can use number of songs as a proxy and compute the metric as number of songs it took for the first skip.

2. *Sustained dissatisfaction:* we assume that the user is dissatisfied in a sustained manner if they skip 3 songs consecutively.

3. *Session coherence:* we define coherence as how similar the recommended musical tracks are. We can use the acoustic_vector of the music tracks to calculate the similarity.

In [20]:
# implement session metric 1: time to first skip (number of songs to first skip)
time_to_skip_1 = log.assign(new=log['session_position'].where(log['skip_1'].eq(True)))\
                    .groupby('session_id').agg({'new':'first', 'session_position':'size'}).reset_index()

time_to_skip_1['new'].mean()

3.9746365276451234

In [13]:
def get_first_skip(df):
  if all(df['skip_1']==False):
        return len(df)
  return df.reset_index(drop=True).query('skip_1==True').index.min()

metrics_time_first_skip_5 = calc_topk_metrics(topk=5, metric_fn=get_first_skip)
metrics_time_first_skip_10 = calc_topk_metrics(topk=10, metric_fn=get_first_skip)
print("Time to first skip NDCG@k, with k= 5 is: ", metrics_time_first_skip_5.mean())
print("Time to first skip NDCG@k, with k= 10 is: ", metrics_time_first_skip_10.mean())

NDCG@k, with k= 5 is:  2.3585
NDCG@k, with k= 10 is:  3.2057


In [20]:
def get_all_skip(df):
  return df.reset_index(drop=True).query('skip_1==True' and 'skip_2==True' and 'skip_3==True').index.min()

metrics_time_all_skip_5 = calc_topk_metrics(topk=5, metric_fn=get_all_skip)
metrics_time_all_skip_10 = calc_topk_metrics(topk=10, metric_fn=get_all_skip)
print("Sustained dissatisfaction, NDCG@k, with k= 5 is: ", metrics_time_all_skip_5.mean())
print("Sustained dissatisfaction, NDCG@k, with k= 10 is: ", metrics_time_all_skip_10.mean())

NDCG@k, with k= 5 is:  1.7121
NDCG@k, with k= 10 is:  1.8181


In [32]:
# implement session metric 2: sustained dissatisfaction: proportions of sessions with 3 consecutive skips
def get_sustained_dis_all_skip(df):
    if any(df['skip_1'].rolling(3).sum()>=3) or any(df['skip_2'].rolling(3).sum()>=3) or any(df['skip_3'].rolling(3).sum()>=3):
        return True
    return False

metrics_sus_dis_all_skip_5 = calc_topk_metrics(topk=5, metric_fn=get_sustained_dis_all_skip)
metrics_sus_dis_all_skip_10 = calc_topk_metrics(topk=10, metric_fn=get_sustained_dis_all_skip)

print("Sustained dissatisfaction@k, with k= 5 is: ", metrics_dis_5.mean())
print("Sustained dissatisfaction@k, with k= 10 is: ", metrics_dis_10.mean())

Sustained dissatisfaction@k, with k= 5 is:  0.5553
Sustained dissatisfaction@k, with k= 10 is:  0.7848


In [44]:
# implement session metric 2: sustained dissatisfaction: proportions of sessions with 3 consecutive skips
log_sus_dis = log.copy()
log_sus_dis['sus_dis'] = log_sus_dis.apply(lambda x: 1 if x['skip_1']==True & x['skip_2']==True & x['skip_3']==True else 0, axis=1)
log_sus_dis_prop = log_sus_dis.groupby(['session_id']).agg({'sus_dis':'sum', 'session_position':'size'}).reset_index()
log_sus_dis_prop['prop'] = log_sus_dis_prop.eval('sus_dis / session_position')
log_sus_dis_prop

,session_id,sus_dis,session_position,prop
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,8,20,0.400000
1,0_0000a72b-09ac-412f-b452-9b9e79bded8f,9,20,0.450000
2,0_00010fc5-b79e-4cdf-bc4c-f140d0f99a3a,12,20,0.600000
3,0_00016a3d-9076-4f67-918f-f29e3ce160dc,14,20,0.700000
4,0_00018b58-deb8-4f98-ac5e-d7e01b346130,1,11,0.090909
...,...,...,...,...
9995,0_0eac164c-f209-4590-8608-a56e67658952,15,20,0.750000
9996,0_0eacbee7-9868-48a0-9ab0-f86069329f50,17,20,0.850000
9997,0_0ead11fc-f32c-4eb4-8fc1-15b51432a404,11,20,0.550000
9998,0_0eae096d-ae4c-4556-8227-0f6294958330,8,12,0.666667


In [45]:
log_sus_dis_prop['prop'].mean()

0.41048890433783625

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def track_similarity(df):
  return np.mean(cosine_similarity(tracks.loc[tracks['track_id'].isin(df['track_id_clean']), [f'acoustic_vector_{i}' for i in range(8)]].values))

track_sim_k5_metrics = calc_topk_metrics(topk=5, metric_fn=track_similarity)
track_sim_k10_metrics = calc_topk_metrics(topk=10, metric_fn=track_similarity)
    
print("Session coherence -- Average track similarity@k, with k= 5 is: ", track_sim_k5_metrics.mean())
print("Session coherence -- Average track similarity@k, with k= 10 is: ", track_sim_k10_metrics.mean())

Average track similarity@k, with k= 5 is:  0.8569095224919191
Average track similarity@k, with k= 10 is:  0.8332746661515774


Once these metrics are implemented, compute these metrics for topk=5 and topk=10 compare their estimates for the production ranker as a correlation plot. Please note which metrics are correlated with ndcg metric.


In [33]:
metrics_ndcg_10.rename('ndcg').to_frame()\
.join(metrics_time_first_skip_10.rename('time_to_first_skip'))\
.join(metrics_sus_dis_all_skip_10.rename('sustained_dissatisfaction'))\
.join(track_sim_k10_metrics.rename('track_sim'))\
.corr()['ndcg']

ndcg                         1.000000
time_to_first_skip           0.592423
sustained_dissatisfaction   -0.280678
track_sim                    0.016455
Name: ndcg, dtype: float64


**Additional goal:**
Implement another simple ranking logic, and compare the performance of both the production ranker and new ranking policy on the ndcg and three behavioral metrics.
A simple ranking policy could include sortby track popularity, or sort by danceability score for the track.

Please report the performance of these rankers on all four metrics.

In [34]:
log_w_pop = pd.merge(log, tracks.loc[:, ['track_id', 'danceability']], 
                     left_on='track_id_clean', 
                     right_on='track_id')

def calc_new_topk_metrics(topk, metric_fn):
    return log_w_pop.groupby('session_id')\
            .apply(lambda x: x.nlargest(topk, ['danceability']))\
            .reset_index(drop = True)\
            .groupby("session_id")\
            .apply(metric_fn)

In [35]:
print("NDCG@k, with k= 5 is: ", calc_new_topk_metrics(topk=5, metric_fn=get_ndcg).mean())
print("NDCG@k, with k= 10 is: ", calc_new_topk_metrics(topk=10, metric_fn=get_ndcg).mean())

NDCG@k, with k= 5 is:  0.7979021412516348
NDCG@k, with k= 10 is:  0.82083829975423


In [36]:
print("Time/Number of songs to first skip@k, with k= 5 is: ", calc_new_topk_metrics(topk=5, metric_fn=get_first_skip).mean())
print("Time/Number of songs to first skip@k, with k= 10 is: ", calc_new_topk_metrics(topk=10, metric_fn=get_first_skip).mean())

Numer of songs to first skip@k, with k= 5 is:  1.7775
Numer of songs to first skip@k, with k= 10 is:  2.4398


In [37]:
print("Sustained dissatisfaction@k, with k= 5 is: ", calc_new_topk_metrics(topk=5, metric_fn=get_sustained_dis_all_skip).mean())
print("Sustained dissatisfaction@k, with k= 10 is: ", calc_new_topk_metrics(topk=10, metric_fn=get_sustained_dis_all_skip).mean())

Sustained dissatisfaction@k, with k= 5 is:  0.4813
Sustained dissatisfaction@k, with k= 10 is:  0.6917


In [38]:
print("Average track similarity@k, with k= 5 is: ", calc_new_topk_metrics(topk=5, metric_fn=track_similarity).mean())
print("Average track similarity@k, with k= 10 is: ", calc_new_topk_metrics(topk=10, metric_fn=track_similarity).mean())

Average track similarity@k, with k= 5 is:  0.8905074592807043
Average track similarity@k, with k= 10 is:  0.8547617305867178


# Part B: Off Policy Evaluation

We log listener behavior based on the recommendations that the production recommender serves to the listener. Using this data to assess any new recommender system, however, can present challenges – the production recommender and the new recommender can drastically differ in the results that they display to the user. For example, maybe the new recommender presents a lot of niche content, while the production recommender presents a lot of popular options. This can be an issue when evaluating a new recommender – If you don’t have any feedback on a recommendation because you never presented it to a user, how can you evaluate whether it’s a good recommendation?
If you have a new policy to test that’s very similar to your old approach, then this won’t be an issue, and it’ll be easy to test! However, if the policy is very different, then you’ll need to collect special logged data.

In this part of the project, we will simulate a recommendation policy and leverage counterfactual estimators as metrics to compare performance.


Lets first begin by generating a few users and products. For ease of simulation, we assume users derive equal satisfaction from each item.

In [39]:

users = np.array(["user1", "user2", "user3"])
products = np.array(
    [
        "product_a",
        "product_b",
        "product_c",
        "product_d",
        "product_e",
        "product_f",
        "product_g",
    ]
)

satisfaction = {
    "product_a": 100,
    "product_b": 150,
    "product_c": 100,
    "product_d": 200,
    "product_e": 500,
    "product_f": 120,
    "product_g": 160,
}



Lets also implement whether a given user will accept a given recommendation or not. Once done, we can implement a target policy that makes recommendations.

In [40]:

def will_purchase(user, product):
    if user == "user1" and (
        product == "product_a" or product == "product_b" or product == "product_c"
    ):
        return True
    elif user == "user2" and (product == "product_d" or product == "product_e"):
        return True
    elif user == "user3" and (product == "product_f" or product == "product_g"):
        return True
    else:
        return False


def choose_user():
    return np.random.choice(users, size=1)


def logging_policy():
    return np.random.choice(products, size=1), 1 / len(products)


class TargetPolicy:
    def __init__(self):
        self.user_probs = {
            "user1": np.array([0.1, 0.1, 0.2, 0.1, 0.15, 0.15, 0.20]),
            "user2": np.array([0.1, 0.10, 0.05, 0.25, 0.3, 0.1, 0.1]),
            "user3": np.array([0.06, 0.06, 0.3, 0.06, 0.06, 0.4, 0.06]),
        }

        for user, probs in self.user_probs.items():
            assert probs.sum() == 1
            assert len(probs) == len(products)

    def recommend(self, user):
        user_prob = self.user_probs[user]
        product = np.random.choice(products, size=1, p=user_prob)
        product_idx = np.where(products == product)
        prob = user_prob[product_idx]

        return product, prob

    def get_prob(self, user, product):
        user_prob = self.user_probs[user]
        product_idx = np.where(products == product)
        product_prob = user_prob[product_idx]

        return product_prob



Having defined all key components of the dataset generation, lets create logged data that we can finally use for evaluation purposes:

In [41]:
def compute_satisfaction(user, product):
    if will_purchase(user, product):
        return satisfaction[product.item()]
    else:
        return 0


def create_logs(n=1000):
    logs = []
    target_policy = TargetPolicy()

    for _ in range(n):
        user = choose_user()

        logging_product, logging_prob = logging_policy()
        model_prob = target_policy.get_prob(user.item(), logging_product)

        target_product, _ = target_policy.recommend(user.item())

        logging_satisfaction = compute_satisfaction(user, logging_product)
        target_satisfaction = compute_satisfaction(user, target_product)

        log = OrderedDict(
            {
                "user_features": user.item(),
                "item_placed": logging_product.item(),
                "item_prob": logging_prob,
                "item_satisfaction": logging_satisfaction,
                "model_prob": model_prob.item(),
                "ab_test_satisfaction": target_satisfaction,
            }
        )

        logs.append(log)

    return pd.DataFrame(logs)

Here is what ur logged data now looks like:

In [42]:
logs = create_logs(n=1000)
logs.head(5)

,user_features,item_placed,item_prob,item_satisfaction,model_prob,ab_test_satisfaction
0,user2,product_a,0.142857,0,0.10,200
1,user3,product_d,0.142857,0,0.06,120
2,user2,product_b,0.142857,0,0.10,500
3,user2,product_g,0.142857,0,0.10,0
4,user3,product_c,0.142857,0,0.30,120


In [43]:
logs[["user_features", "item_placed", "item_prob", "item_satisfaction"]].sample(n=10)

,user_features,item_placed,item_prob,item_satisfaction
302,user3,product_b,0.142857,0
969,user1,product_a,0.142857,100
231,user3,product_b,0.142857,0
270,user2,product_f,0.142857,0
464,user1,product_a,0.142857,100
937,user2,product_d,0.142857,200
453,user3,product_d,0.142857,0
901,user2,product_e,0.142857,500
306,user1,product_d,0.142857,0
197,user2,product_g,0.142857,0


With all the dataset ready, lets compute the mean rewards (satisfaction) for the logging/production policy and the target policy:

In [44]:
%%time
sim = create_logs(n=100000)
logging_policy = sim["item_satisfaction"].mean()
target_policy = sim["ab_test_satisfaction"].mean()

print(f"Expected reward from logging policy: {logging_policy: .2f}")
print(f"Expected reward from target policy: {target_policy: .2f}")

Expected reward from logging policy:  63.20
Expected reward from target policy:  100.52
CPU times: user 14.5 s, sys: 525 ms, total: 15.1 s
Wall time: 16 s


Now lets implement the IPS estimator:

In [45]:
def compute_ips(df):
    assert {"model_prob", "item_prob", "item_satisfaction"}.issubset(df.columns)
    return (df["model_prob"] / df["item_prob"] * df["item_satisfaction"]).mean()

In [46]:
ips_est = compute_ips(logs)
ips_est

111.804

Computing the IPS estimator on our 1,000 entry log gives an average revenue of 109.34 (very close to the true performance of 100.99) compared with the average revenue of the logging policy of 63.36. Therefore, we should be confident to deploy our target policy to production and do an AB test comparing it with the logging policy as a final validation.

## Goals for this part of project:

Finish the implementation of two additional off-policy estimators:
1. Capped IPS
2. Normalized Capped Importance Sampling (NCIS)

Feel free to try different capping thresholds, and compare the reward and standard deviations of these estimators with the IPS estimator and mean reward.


In [47]:
def compute_capped_ips(df, cap=1000):
    return ((df["model_prob"] / df["item_prob"]).apply(lambda x: min(x, 1000)) * df["item_satisfaction"]).mean() 

compute_capped_ips(logs)

111.804

In [49]:
def compute_ncis(df, cap=1000):
  return ((df["model_prob"] / df["item_prob"]).apply(lambda x: min(x, 1000)) * df["item_satisfaction"]).mean()/(df["model_prob"] / df["item_prob"]).mean()

compute_ncis(logs)

112.05275712080818